# Technical Demo

In [1]:
# import
from pathlib import Path
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama

from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader                                                                                                                                                                                                                                                                                                                                                                                                                                      
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

In [3]:
llm = Ollama(model="llama2:7b-chat-q4_K_M")

Prompting the LLM with Langchain

1. Create a prompt from `ChatPromptTemplate` or other similar classes
2. Use '|' separator as the symbol to create a chain object
3. call chain.invoke() and pass the words to be formated as a dictionary to prompt the LLM

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me about history of {topic}")
chain = prompt | llm

In [9]:
chain.invoke({"topic":"Blockchain"})

'\nBlockchain, the technology behind cryptocurrencies like Bitcoin and Ethereum, has a rich and fascinating history that spans over two decades. Here is a brief overview of the key events and milestones in the history of blockchain:\n\n1998: The concept of blockchain was first introduced by Stuart Haber and W. Scott Stornetta, two computer scientists who proposed a system called "Paper-based Digital Cash" that used a chain of blocks to record transactions. However, their idea did not gain much traction at the time.\n\n2008: The Bitcoin whitepaper was published by an anonymous individual or group using the pseudonym Satoshi Nakamoto. This document outlined the design and functionality of a decentralized digital currency called Bitcoin, which would use blockchain technology to record transactions and manage the creation of new units.\n\n2009: The first block in the Bitcoin blockchain was mined by Satoshi Nakamoto on January 3rd. This marked the beginning of the Bitcoin network and the st

RAG

In [4]:
# load the document and split it into chunks
doc_path = Path("reviews/cyberpunk_2077_phantom_liberty_01.txt")

loader = TextLoader(str(doc_path))
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

query = "Generate a summary of the document."
docs = db.similarity_search(query)

print(docs[0].page_content)

/root/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


It took me a second to come around to Phantom Liberty's set-up: rescue the president of the New United States after Space Force One (lol) gets shot down over Dogtown, a newly added district of Night City that looks like the Thunderdome by way of Blade Runner: 2049's haunting, irradiated Las Vegas.

CD Projekt is great at teeing up a story that seems like it'd be derivative or Spike TV-edgy, then absolutely curving you with unexpected depth and nuance, and that's no different here. President Myers comes off as a down to earth, "one of the guys" former soldier, like Harrison Ford from Air Force One, but there's a serpentlike ambition to her that comes out as the story goes on.


In [25]:
# make a chain

from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    return_source_documents=True
)

In [38]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.page_content)
        print(source.metadata['source'])
        print('-'*10)

In [39]:
# full example
query = "Generate a summary of the document."
llm_response = chain.invoke(query)
process_llm_response(llm_response)

print()
print('-'*20)
print(llm_response)

The document is a review of the Phantom Liberty expansion for Cyberpunk 2077, which provides a new storyline, characters, and gameplay experiences for players to enjoy. The reviewer notes that the expansion is a "splendid sendoff" for the game's world, which CD Projekt has been working on for over a decade. They praise the expansion's writing and world-building, stating that it hangs with the big dogs when it comes to cinematic storytelling.

The reviewer highlights several of the gigs in the expansion, including rescuing an underworld doctor, infiltrating a youth sports academy, and helping out two hapless detectives. They describe these tasks as having a "slapstick absurdity" that they enjoyed.

The main players in the expansion all think they're doing the right thing, but ultimately, the player must make a difficult choice that will determine the outcome of the story. The reviewer notes that both ending paths feature their own triumphs and gut punches, and they chose the path with m